# General

## Imports

In [ ]:
from collections import deque
from lxml import etree
from pathlib import Path
from pathlib import PurePosixPath
from datetime import datetime
import os
import pandas as pd
import pickle
import pprint
import random

## Parameters

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = "../data/db/mom-data/metadata.charter.public"
#directoryPath = "../data/db/mom-data/metadata.charter.public/AT-WStLA/HAUrk"
fileExtension = ".cei.xml"

# Create .xml mapping

In [ ]:
# to do, see mapping.ipynb

# Load .xml mapping

In [ ]:
with open('../data/mapping/CEI2CSV.pkl', 'rb') as f:
    mapping = pickle.load(f)

pprint.pprint(mapping)

# Scan directory and create paths

In [ ]:
def get_file_paths(directory):
    for entry in os.scandir(directory):
        if entry.is_file() and entry.name.endswith(fileExtension):
            yield Path(entry.path)
        elif entry.is_dir():
            yield from get_file_paths(entry.path)
        else:
            continue

In [ ]:
paths = [f"{PurePosixPath(path)}" for path in get_file_paths(directoryPath)]

In [ ]:
paths_sample = random.sample(paths, 100)

# Create deques

In [ ]:
deques = deque(["atom_id", "cei_placeName", "cei_lang_MOM", "cei_tenor", "cei_date", "cei_date_ATTRIBUTE_value", "cei_dateRange", "cei_dateRange_ATTRIBUTE_from", "cei_dateRange_ATTRIBUTE_to", "cei_abstract", "cei_abstract_foreign", "cei_graphic_ATTRIBUTE_url_orig", "cei_graphic_ATTRIBUTE_url_copy"])
atom_id, cei_placeName, cei_lang_MOM, cei_tenor, cei_date, cei_date_ATTRIBUTE_value, cei_dateRange, cei_dateRange_ATTRIBUTE_from, cei_dateRange_ATTRIBUTE_to, cei_abstract, cei_abstract_foreign, cei_graphic_ATTRIBUTE_url_orig, cei_graphic_ATTRIBUTE_url_copy = ([] for i in range(len(deques)))

# Query paths and fill deques

In [ ]:
def get_xpath_result(xpath_mapping):
    return tree.xpath(xpath_mapping, namespaces = namespaces, smart_strings = False)

for file in paths_sample:
    with open(file, "r", encoding="utf-8") as f:
        tree = etree.parse(file)
        atom_id.append(get_xpath_result(f"{mapping['atom:id']}/text()"))
        cei_abstract.append("".join(get_xpath_result(f"{mapping['cei:abstract']}/descendant-or-self::text()[not(self::cei:sup)]")))
        cei_abstract_foreign.append(get_xpath_result(f"{mapping['cei:foreign']}/text()"))
        cei_tenor.append("".join(get_xpath_result(f"{mapping['cei:tenor']}/descendant-or-self::text()[not(self::cei:sup)]")))
        cei_placeName.append(get_xpath_result(f"{mapping['cei:issued/cei:placeName']}/text()"))
        cei_lang_MOM.append(get_xpath_result(f"{mapping['cei:lang_MOM']}/text()"))
        cei_date.append(get_xpath_result(f"{mapping['cei:date']}/text()"))
        cei_dateRange.append(get_xpath_result(f"{mapping['cei:dateRange']}/text()"))
        cei_date_ATTRIBUTE_value.append(get_xpath_result(f"{mapping['cei:date']}/@value"))
        cei_dateRange_ATTRIBUTE_from.append(get_xpath_result(f"{mapping['cei:dateRange']}/@from"))
        cei_dateRange_ATTRIBUTE_to.append(get_xpath_result(f"{mapping['cei:dateRange']}/@to"))
        cei_graphic_ATTRIBUTE_url_orig.append(get_xpath_result(f"{mapping['@url']}"))
        cei_graphic_ATTRIBUTE_url_copy.append(get_xpath_result(f"{mapping['cei:graphic/@url']}"))



# Merge deques to dataframe

In [ ]:
# generate code part for renaming frames
for i, item in enumerate(deques):
    print(f"{i}:'{item}',")
    i =+ 1

In [ ]:
contents = list(zip(atom_id, cei_placeName, cei_lang_MOM, cei_tenor, cei_date, cei_date_ATTRIBUTE_value, cei_dateRange, cei_dateRange_ATTRIBUTE_from, cei_dateRange_ATTRIBUTE_to, cei_abstract, cei_abstract_foreign, cei_graphic_ATTRIBUTE_url_orig, cei_graphic_ATTRIBUTE_url_copy))
contents_full = pd.DataFrame(contents).rename(
    columns={ 0:'atom_id',
1:'cei_placeName',
2:'cei_lang_MOM',
3:'cei_tenor',
4:'cei_date',
5:'cei_date_ATTRIBUTE_value',
6:'cei_dateRange',
7:'cei_dateRange_ATTRIBUTE_from',
8:'cei_dateRange_ATTRIBUTE_to',
9:'cei_abstract',
10:'cei_abstract_foreign',
11:'cei_graphic_ATTRIBUTE_url_orig',
12:'cei_graphic_ATTRIBUTE_url_copy' })

In [ ]:
contents_full

# Export dataframe

In [ ]:
timemarker = datetime.today().strftime('%Y-%m-%d-%H%M')
contents_full.to_json(f"../data/output/charters_{timemarker}.json")
contents_full.to_parquet(f"../data/output/charters_{timemarker}.parquet")

# Import generated data as dataframe

In [ ]:
contents_full_json = pd.read_json("../data/output/x.json")
contents_full_parquet = pd.read_parquet('../data/output/x.parquet', engine='pyarrow')